In [12]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
from pathlib import Path

In [15]:
radio_dir = Path.home() / 'Desktop/insight_fellows/radio-project-tbd'
df = pd.read_csv(radio_dir / 'data/radio_list_preprocessed_wLanguages.csv', index_col = 0)
df

,call sign,owner,format,lat,long,station name,power,p. unit,frequency,city of licence,n_stations,language,genre
0,CJKX-FM,Durham Radio,country,43.85833,-79.03639,KX 96,50,kW,95.9 FM,Ajax,1,1,country
1,CJNK-FM,Weatheradio Canada,weather alerts,45.58417,-78.35833,Weatheradio,50,w,100.1 FM,Algonquin Park,2,1,weather alerts
2,CJNK-FM-1,Weatheradio Canada,weather alerts,45.58417,-78.35833,Weatheradio,22,w,101.3 FM,Algonquin Park East,1,1,weather alerts
3,CJNK-FM-2,Weatheradio Canada,weather alerts,45.58417,-78.35833,Weatheradio,22,w,101.3 FM,Algonquin Park West,1,1,weather alerts
4,CFOA-FM,Friends of Algonquin Park,tourist/park information,45.58417,-78.35833,Algonquin Park,10,w,102.7 FM,Algonquin Park,2,1,tourist/park information
...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,CKNX-FM,Blackburn Radio,adult contemporary,43.88778,-81.31167,101.7 The One,100,kW,101.7 FM,Wingham,4,1,adult contemporary
532,CJFH-FM,Sound of Faith Broadcasting,Christian radio,43.13056,-80.74667,Hope FM,37,w,94.3 FM,Woodstock,3,1,christian radio
533,CKDK-FM,Corus Entertainment,country,43.13056,-80.74667,Country 104,51,kW,103.9 FM,Woodstock,3,1,country
534,CIHR-FM,Byrnes Communications,adult contemporary,43.13056,-80.74667,Heart FM,20,kW,104.7 FM,Woodstock,3,1,adult contemporary


In [36]:
station_counts = df['city of licence'].value_counts().reset_index()
station_counts.columns = ['city of licence', 'n_stations']
sorted_locations = station_counts.sort_values(by = 'n_stations', ascending = False)['city of licence']

In [44]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Which stations are around you?"),
    dcc.Graph(id='graph'),
    html.Label([
        "What do you want to listen to?",
        dcc.Dropdown(
            id='genre-dropdown', clearable=False,
            value=None, options=[
                {'label': g, 'value': g}
                for g in df.genre.unique()
            ])
    ]),
    html.Br(),
    html.Label([
        "Where are you located?",
        dcc.Dropdown(
            id='location-dropdown', clearable=False,
            value=None, options=[
                {'label': g, 'value': g}
                for g in sorted_locations
            ])
    ]),
    html.Br(),
    html.Div([], id = 'stations4user')
])
# Define callback to update graph
@app.callback(
    [Output('graph', 'figure'),
     Output('stations4user', 'children')],
    [Input("genre-dropdown", "value"),
    Input("location-dropdown", "value")]
)
def update_figure(genre, location):
    data4genre = df[(df.genre == genre) & (df['city of licence'] == location)].copy()
    stations = ', '.join(data4genre['station name'].values)
    fig = go.Figure(data=go.Scattergeo(
            lon = data4genre['long'],
            lat = data4genre['lat'],
            text = stations,
            mode = 'markers',
            marker_size = data4genre.shape,
            ))
    
    fig.update_layout(
            #geo_scope='north america',
            geo = dict(
                # adjusted for Ontario
                lataxis = dict(range=[40,60]),
                lonaxis = dict(range=[-100,-60])),
        )
    return fig, 'Your safest bets are ' + stations
# Run app and display result inline in the notebook
app.run_server(mode='inline')